<a href="https://colab.research.google.com/github/O-Kpy/Kaggle/blob/main/Kaggle_Contradictory%2C_My_Dear_Watson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contradictory, My Dear Watson

Detecting contradiction and entailment in multilingual text **using TPUs**

- TPU : 구글에서 2016년 5월에 발표한 데이터 분석 및 딥러닝용 하드웨어이다. 벡터/행렬연산의 병렬처리에 특화되어 있으며 넘사벽급의 전성비를 자랑한다.

In [1]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev
!pip install transformers

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  35041      0 --:--:-- --:--:-- --:--:-- 35041
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
Found existing installation: torch 1.9.0+cu102
     |████████████████████████████████| 57 kB 2.3 MB/s 
Uninstalling torch-1.9.0+cu102:
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.272 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatibl

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import torch_xla
import torch_xla.core.xla_model as xm

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold

import transformers
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification
from transformers import XLMRobertaConfig, XLMRobertaForSequenceClassification, XLMRobertaTokenizer
from transformers import AdamW

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import torch_xla
import torch_xla.core.xla_model as xm

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold

import transformers
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import AdamW

import warnings
warnings.filterwarnings('ignore')

print(torch.__version__)

1.6.0a0+bf2bbd9


# Bert model 

---



==> Bidirectional(양방향 모델) : 왼쪽에서 오른쪽으로, 오른쪽에서 왼쪽으로 모두 텍스트를 읽을 수 있음을 의미합니다.

In [ ]:
# Bert Vocabulary

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
len(tokenizer.vocab)

30522

In [ ]:
# token의 구성
bert_vocab = tokenizer.vocab

print(bert_vocab['[CLS]'])
print(bert_vocab['[SEP]'])
print(bert_vocab['[PAD]']) # padding=0

# 문자열 토큰화
print(bert_vocab['hello'])
print(bert_vocab['world'])

bert_keys = []

for token in tokenizer.vocab.keys():
  bert_keys.append(token)

print(bert_keys[101])
print(bert_keys[102])
print(bert_keys[0])

print(bert_keys[7592])
print(bert_keys[2088])

101
102
0
7592
2088
[CLS]
[SEP]
[PAD]
hello
world


In [ ]:
# Bert model 구성요소

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased',
                                                      num_labels=3,  # 결과 라벨 수
                                                      output_attentions=False,
                                                      output_hidden_states=False)

outputs = model(input_ids=b_input_ids,
                token_type_ids=b_token_type_ids,
                attention_mask=b_attention_mask,
                labels=b_labels)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

NameError: ignored

1. input_ids
 - 토큰으로 표시되는 문장(토큰)
 
 [CLS] : classifier token, vlaue:101

 [SEP] : separator token, value:102

 [PAD] : padding token, value:0

  For one sentence as input:
    - [CLS] ...word tokens... [SEP]

  For two sentences as input:
    - [CLS] ...sentence1 tokens... [SEP]..sentence2 tokens... [SEP]

ex) [101[CLS], 7592, 2045, 1012,  102[SEP],    0,    0,    0,    0,    0]

2. token_type_ids
 - 입력의 일부가 되어야 하는 두 문장이 있을 때 사용
 - [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
 - 0은 첫번째 sentence 식별
 - 1은 두번째 sentence 식별
 - 오른쪽 0은 padding

3. attantion_mask
 - attention_mask의 길이는 input_ids의 길이와 같아야 한다.
 - 모델에 input_ids의 어떤 토큰이 워드이고 어떤 토큰이 패딩인지 알려준다.

ex)
 - Tokens: [101, 7592, 2045, 1012,  102,    0,    0,    0,    0,    0]
 - Attention mask: [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

4. labels
 - 각 행의 label(target)
 - ex) [0, 2, 0, 1, 2, 0, 3, 1]

---

# how to use tokenizer to auto format input data

In [ ]:
# encode
max_length = 10
sentence1 = 'hello there.'

encoded_dict = tokenizer.encode_plus(
    sentence1,  # text to tokens
    add_special_tokens=True,  # [CLS], [SEP]를 추가
    max_length=max_length,
    truncation=True,  # # Pad or truncate.
    return_attention_mask=True,
    return_tensors='pt'
)

encoded_dict

{'input_ids': tensor([[ 101, 7592, 2045, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [ ]:
input_ids = encoded_dict['input_ids'][0]
token_type_ids = encoded_dict['token_type_ids'][0]
attention_mask = encoded_dict['attention_mask'][0]

print(input_ids)
print(token_type_ids)
print(attention_mask)

tensor([ 101, 7592, 2045, 1012,  102])
tensor([0, 0, 0, 0, 0])
tensor([1, 1, 1, 1, 1])


In [ ]:
# decode
a = tokenizer.decode(input_ids,
                     skip_special_tokens=False)

b = tokenizer.decode(input_ids,
                     skip_special_tokens=True)

print(a)
print(b)

[CLS] hello there. [SEP]
hello there.


# XLM-RoBERTa Vocabulary

- XLM은 교차 언어 모델을 의미. 

- XLM-Roberta(XLM-R)는 다국어 BERT를 능가하는 사전 훈련된 **다국어 모델**이다.
 - XLM-R이 훨씬 더 많은 데이터를 사용하여 교육을 받았기 때문입니다. (XLM-R은 또한 100개의 언어로 train 되었다.)

1. input_ids
 - BERT의 토큰화와 비슷
    - bos_token_id (s) : 0
    - eos_token_id (/s) : 2
    - sep_token_id (/s) : 2
    - pad_token_id (pad) : 1

 - For one sentence as input:
    - (s) ...word tokens... (/s)

 - For two sentences as input:<br>
    - (s) ...sentence1 tokens... (/s)(/s)..sentence2 tokens... (/s)

 - padding은 1
    - ex)[[0, 35378, 2685, 5, 2, 1, 1, 1, 1, 1]]


2. token_type_ids
 - XLM-RoBERTa는 token_type_ids를 사용하지 않는다.

3. attention_mask
 - BERT와 같다.

4. labels
 - target 레이블(BERT와 같다.)


In [4]:
! pip install sentencepiece

     |████████████████████████████████| 1.2 MB 4.9 MB/s 


In [2]:
model_type='xlm-roberta-base'
tokenizer = XLMRobertaTokenizer.from_pretrained(model_type)

In [3]:
print(tokenizer.vocab_size)  # check vocab size
tokenizer.special_tokens_map # what are the special tokens

250002


{'bos_token': '<s>',
 'cls_token': '<s>',
 'eos_token': '</s>',
 'mask_token': '<mask>',
 'pad_token': '<pad>',
 'sep_token': '</s>',
 'unk_token': '<unk>'}

In [ ]:
# xlm-roberta-base model

model_type = 'xlm-roberta-base'

model = XLMRobertaForSequenceClassification.from_pretrained(model_type,
                                                            num_labels=3)

In [4]:
max_len = 10

text = '너의 이름은?'

encoded_dict = tokenizer.encode_plus(text,
                                     add_special_tokens=True,
                                     max_length=max_len,
                                     truncation=True,
                                     return_attention_mask=True,
                                     return_tensors='pt')
encoded_dict

{'input_ids': tensor([[    0, 70204,   367, 50932,   697,    32,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [11]:
decode = tokenizer.decode(encoded_dict['input_ids'][0],
                          skip_special_tokens=False)

print(f"토큰화 된 것:{encoded_dict['input_ids'][0]}")
print(f'토큰화 안된것(디코드 한 것):{decode}')

토큰화 된 것:tensor([    0, 70204,   367, 50932,   697,    32,     2])
토큰화 안된것(디코드 한 것):<s> 너의 이름은?</s>


In [ ]:
# encode
max_len = 10

sentence1 = 'hello there.'

encoded_dict = tokenizer.encode_plus(sentence1,
                                     add_special_tokens=True,
                                     max_length=max_len,
                                     truncation=True,
                                     return_attention_mask=True,
                                     return_tensors='pt')

encoded_dict
# input_ids, attention_mask로 구성 -- Bert는 input_ids, attention_mask, token_type_ids 구성

{'input_ids': tensor([[    0, 33600,    31,  2685,     5,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [ ]:
input_ids = encoded_dict['input_ids'][0]
att_mask = encoded_dict['attention_mask'][0]

print(input_ids)
print(att_mask)

tensor([    0, 33600,    31,  2685,     5,     2])
tensor([1, 1, 1, 1, 1, 1])


In [ ]:
# decode
a = tokenizer.decode(input_ids,
                     skip_special_tokens=False)  # 'bos_token': '<s>', 'cls_token': '<s>', 'eos_token': '</s>', 'mask_token': '<mask>', 'pad_token': '<pad>', 'sep_token': '</s>', 'unk_token': '<unk>' 등

b = tokenizer.decode(input_ids,
                     skip_special_tokens=True)

print(a)
print(b)

<s> hello there.</s>
hello there.


# Contradictory, My Dear Watson 실습 Start

In [12]:
train = pd.read_csv('/content/drive/MyDrive/dataset/kaggle/nlp/contradictory-my-dear-watson/contradictory-my-dear-watson-train.csv')
test = pd.read_csv('/content/drive/MyDrive/dataset/kaggle/nlp/contradictory-my-dear-watson/contradictory-my-dear-watson-test.csv')
submit = pd.read_csv('/content/drive/MyDrive/dataset/kaggle/nlp/contradictory-my-dear-watson/contradictory-my-dear-watson-sample_submission.csv')
train.shape, test.shape, submit.shape

((12120, 6), (5195, 5), (5195, 2))

In [13]:
train.head()
# XLM-RoBERTa 쓰기 딱 좋구만...

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [14]:
df = shuffle(train)

sk = StratifiedKFold(n_splits=5, shuffle=True, random_state=1024)

y_train = df['label']

fold_list = list(sk.split(df, y_train))

train_list = []
val_list = []

for i, fold in enumerate(fold_list):
  df_train = df[df.index.isin(fold[0])]
  df_val = df[df.index.isin(fold[1])]

  train_list.append(df_train)
  val_list.append(df_val)

print(f'train length : {len(train_list)}')
print(f'val length : {len(val_list)}')

train length : 5
val length : 5


In [ ]:
# 위에 하고 같음
list_train = []
list_val = []
for i, (train_index, val_index) in enumerate(sk.split(df, y_train)):
  train_df, val_df = df.iloc[train_index], df.iloc[val_index]

  list_train.append(train_df)
  list_val.append(val_df)

In [15]:
train_list[0]

,id,premise,hypothesis,lang_abv,language,label
3865,ca5dea25ad,"Έτσι, μου πήρε περίπου, μια ώρα με δύο ώρες μό...",Μου πήρε αρκετόχ ρόνο να το βρω.,el,Greek,0
462,253441c9c4,yeah i mean this this Escort even when the hea...,"Once the head gasket went out, the Escort stop...",en,English,2
4081,25f743dd41,Tena nyanyangu akaamka na akaanza kutembea chi...,Nyanya alikaa kwenye ukumbi.,sw,Swahili,2
2270,a52cf9f3de,Ήταν ένα μαύρο άτομο με ανοιχτόχρωμο δέρμα.,Έχει απίστευτα σκούρο δέρμα.,el,Greek,2
6659,fff12e05cf,"By then, the program had added Carroll and Gra...",Carroll and Grayson counties had been added.,en,English,0
...,...,...,...,...,...,...
8742,d56589e51c,یہ آج ہم زندہ راستہ ہے، گیری کہہ رہا ہے، کیوں ...,-گہری کو خوشی کی فکر نہیں,ur,Urdu,2
6615,e65978db8f,لیکن اس کے علاوہ میں امید کرتا ہوں کہ یہ ابھی ...,میں امید کرتا ہوں کہ موسم مزید سرد نہ ہو اور ت...,ur,Urdu,0
6396,946faaf408,"Finally, the FDA will conduct workshops, issue...",The FDA is set to conduct workshops.,en,English,0
6187,a5757ae732,ในฐานะที่เป็นโปรเตสแตนต์ Pierre du Calvet ได้ร...,ปีแยร์ไม่เคยถูกจับ,th,Thai,2


In [16]:
val_list[0]

,id,premise,hypothesis,lang_abv,language,label
11833,68b14c67b2,From ethnic food shops and vintage clothing st...,There are so many places to shop that you can ...,en,English,0
7890,3663a23e87,"Nein, sie sind, sie sind immer noch auf Tour; ...",Sie lieben es zu reisen.,de,German,1
4568,0147bb9de8,Very well ”but it's all extremely mysterious. ...,Poirot went into the Analytical Chemist.,en,English,0
11274,ec02ba3e97,well i i'm doing computer science computer eng...,I am currently majoring in Biology at school.,en,English,2
11069,fdf9081d01,"Last year, they were spooked.",They were spooky.,en,English,2
...,...,...,...,...,...,...
7775,5979f0b39d,Sixty percent of Americans are frustrated and ...,A majority of Americans are not happy with the...,en,English,0
7434,f19cfef124,"Поскольку все время я стоял на коленях, упирая...",Я кладу голову на дерево.,ru,Russian,0
9495,bd39bb98ca,"और समान रूप से असाधारण रूप से, पिछले कुछ मिलिय...",होमिनिड्स के ईकोनोस्फीयर हर पीढ़ी की जटिलता मे...,hi,Hindi,1
9791,3b5e98b36a,"Politically, it's anti-democratic, replacing c...",It's anti-democratic and takes the decision-ma...,en,English,1


# 토큰화, Dataset, Dataloader

In [17]:
MODEL_TYPE = 'bert-base-multilingual-uncased'

NUM_FOLDS = 5

# Saving 5 TPU models will exceed the 4.9GB disk space.
# Therefore, will will only train on 3 folds.
NUM_FOLDS_TO_TRAIN = 3 

L_RATE = 1e-5
MAX_LEN = 256
NUM_EPOCHS = 3
BATCH_SIZE = 32

In [18]:
device = xm.xla_device()
print(device)

xla:1


In [19]:
# bert-base-multilingual-uncased 토큰
tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE,
                                          do_lower_case=True)

In [23]:
class CompDataset(Dataset):
  def __init__(self, df):
    self.df_data = df
  
  def __getitem__(self, index):
    sentence1 = self.df_data.loc[index, 'premise']
    sentence2 = self.df_data.loc[index, 'hypothesis']

    encoded_dict = tokenizer.encode_plus(sentence1, sentence2,
                                         add_special_tokens=True,
                                         max_length=MAX_LEN,
                                         pad_to_max_length=True,
                                         truncation=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
    
    input_ids = encoded_dict['input_ids'][0]
    attention_mask = encoded_dict['attention_mask'][0]
    token_type_ids = encoded_dict['token_type_ids'][0]

    target = torch.tensor(self.df_data.loc[index, 'label'])

    sample = (input_ids, attention_mask, token_type_ids, target)
    return sample

  def __len__(self):
    return len(self.df_data)


class TestDataset(Dataset):
  def __init__(self, df):
    self.df_data = df

  def __getitem__(self, index):
    sentence1 = self.df_data.loc[index, 'premise']
    sentence2 = self.df_data.loc[index, 'hypothesis']

    encoded_dict = tokenizer.encode_plus(sentence1, sentence2,
                                         add_special_tokens=True,
                                         max_length=MAX_LEN,
                                         pad_to_max_length=True,
                                         truncation=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
    input_ids = encoded_dict['input_ids'][0]
    attention_mask = encoded_dict['attention_mask'][0]
    token_type_ids = encoded_dict['token_type_ids'][0]

    sample = (input_ids, attention_mask, token_type_ids)
    return sample
  
  def __len__(self):
    return len(self.df_data)

In [ ]:
# Dataset으로 tokenizer까지
class CompDataset(Dataset):
  def __init__(self, df):
    self.df_data = df

  def __getitem__(self, index):
    sentence1 = self.df_data.loc[index, 'premise']
    sentence2 = self.df_data.loc[index, 'hypothesis']

    encoded_dict = tokenizer.encode_plus(sentence1, sentence2,
                                         add_special_tokens=True,
                                         max_length=MAX_LEN,
                                         pad_to_max_length=True,
                                         truncation=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
    
    input_ids = encoded_dict['input_ids'][0]
    attention_mask = encoded_dict['attention_mask'][0]
    token_type_ids = encoded_dict['token_type_ids'][0]

    target = torch.tensor(self.df_data.loc[index, 'label'])

    sample = (input_ids, attention_mask, token_type_ids, target)
    return sample

  def __len__(self):
    return len(self.df_data)

class TestDataset(Dataset):
  def __init__(self, df):
    self.df_data = df



  def __getitem__(self, index):
    sentence1 = self.df_data.loc[index, 'premise']
    sentence2 = self.df_data.loc[index, 'hypothesis']

    encoded_dict = tokenizer.encode_plus(sentence1, sentence2,
                                         add_special_tokens=True,
                                         max_length=MAX_LEN,
                                         pad_to_max_length=True,
                                         truncation=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
    
    input_ids = encoded_dict['input_ids'][0]
    attention_mask = encoded_dict['attention_mask'][0]
    token_type_ids = encoded_dict['token_type_ids'][0]

    sample = (input_ids, attention_mask, tokens_type_ids)

    return sample

  def __len__(self):
    return len(self.df_data)

In [ ]:
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df



    def __getitem__(self, index):

        # get the sentence from the dataframe
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        # Process the sentence
        # ---------------------

        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,           # Sentences to encode.
                    add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
                    max_length = MAX_LEN,           # Pad or truncate all sentences.
                    pad_to_max_length = True,
                    truncation=True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',          # Return pytorch tensors.
               )  
        
        # These are torch tensors already.
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        token_type_ids = encoded_dict['token_type_ids'][0]
        
        # Convert the target to a torch tensor
        target = torch.tensor(self.df_data.loc[index, 'label'])

        sample = (padded_token_list, att_mask, token_type_ids, target)


        return sample


    def __len__(self):
        return len(self.df_data)
    
    
    
    
    

class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df



    def __getitem__(self, index):

        # get the sentence from the dataframe
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        # Process the sentence
        # ---------------------

        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,           # Sentence to encode.
                    add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
                    max_length = MAX_LEN,           # Pad or truncate all sentences.
                    pad_to_max_length = True,
                    truncation=True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',          # Return pytorch tensors.
               )
        
        # These are torch tensors already.
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        token_type_ids = encoded_dict['token_type_ids'][0]
               

        sample = (padded_token_list, att_mask, token_type_ids)


        return sample


    def __len__(self):
        return len(self.df_data)

In [21]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [ ]:
# num_workers 쓰기 위한 cpu_count()
NUM_CORES = os.cpu_count()

NUM_CORES

40

In [25]:
train = CompDataset(df_train)
valid = CompDataset(df_val)
test = TestDataset(test)

train_dataloader = torch.utils.data.DataLoader(train,
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               num_workers=4)
val_dataloader = torch.utils.data.DataLoader(valid,
                                             batch_size=BATCH_SIZE,
                                             shuffle=True,
                                             num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test,
                                              batch_size=BATCH_SIZE,
                                              shuffle=True,
                                              num_workers=4)

In [ ]:
train = CompDataset(df_train)
valid = CompDataset(df_val)
test = TestDataset(test)

train_dataloader = torch.utils.data.DataLoader(train,
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               num_workers=4)

val_dataloader = torch.utils.data.DataLoader(valid,
                                             batch_size=BATCH_SIZE,
                                             shuffle=True,
                                             num_workers=4)

test_dataloader = torch.utils.data.DataLoader(test,
                                              batch_size=BATCH_SIZE,
                                              shuffle=False,
                                              num_workers=4)

len(train_dataloader), len(val_dataloader), len(test_dataloader)

(303, 76, 163)

# Define the Model(BertForSequenceClassification)

In [ ]:
# model 불러오기(BertForSequnceClassification)
model = BertForSequenceClassification.from_pretrained(MODEL_TYPE,
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

# Test the Model

In [27]:
train_dataloader = torch.utils.data.DataLoader(train,
                                               batch_size=8,
                                               shuffle=True,
                                               num_workers=1)

batch = next(iter(train_dataloader))

b_input_ids = batch[0].to(device)
b_input_mask = batch[1].to(device)
b_token_type_ids = batch[2].to(device)
b_labels = batch[3].to(device)

b_input_ids.size(), b_input_mask.size(), b_token_type_ids.size(), b_labels.size()

(torch.Size([8, 256]),
 torch.Size([8, 256]),
 torch.Size([8, 256]),
 torch.Size([8]))

# 모델의 output 탐구



In [28]:
# output 생성
outputs = model(b_input_ids, 
                token_type_ids=b_token_type_ids, 
                attention_mask=b_input_mask,
                labels=b_labels)

In [ ]:
 # 하나의 튜플 안에 (loss, predicts)
 len(outputs)

2

In [29]:
# loss
outputs[0]

tensor(1.3256, device='xla:1', grad_fn=<NllLossBackward>)

In [30]:
# predicts
outputs[1]

tensor([[ 0.5050,  0.3764, -0.1197],
        [ 0.4793,  0.3663, -0.0988],
        [ 0.5336,  0.4244, -0.1949],
        [ 0.3303,  0.1908,  0.0662],
        [-0.0418, -0.0736,  0.2190],
        [ 0.1941,  0.0743,  0.1937],
        [ 0.4264,  0.3148, -0.0311],
        [ 0.4691,  0.3251, -0.0583]], device='xla:1', grad_fn=<AddmmBackward>)

In [31]:
preds = outputs[1].detach().cpu().numpy()

y_true = b_labels.detach().cpu().numpy()
y_pred = np.argmax(preds, axis=1)

y_pred

array([0, 0, 0, 0, 2, 0, 0, 0])

In [32]:
acc = accuracy_score(y_true=y_true, y_pred=y_pred)

acc

0.25

# Train the Model

In [52]:
import random
import gc

seed_val = 1024

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

fold_val_acc_list = []
for i in range(0, NUM_FOLDS):
  fold_val_acc_list.append([])

# for each fold
for epoch in range(0, NUM_EPOCHS):
  print("\nNum folds used for training:", NUM_FOLDS_TO_TRAIN)
  print(f'=========== Epoch {epoch+1} / {NUM_EPOCHS} ===========')

  num_folds = len(train_list)

  epoch_acc_scores_list = []

  for fold_index in range(0, NUM_FOLDS_TO_TRAIN):
    print('\n== Fold Model', fold_index)

    if epoch == 0:
      model = BertForSequenceClassification.from_pretrained(MODEL_TYPE,
                                                            num_labels=3,
                                                            output_attentions=False,
                                                            output_hidden_states=False)
      
      model.to(device)
      optimizer = AdamW(model.parameters(),
                        lr=L_RATE,
                        eps=1e-5)
    
    else:
      path_model = 'model_' + str(fold_index) + '.bin'
      model.load_state_dict(torch.load(path_model))

      model.to(device) 
    
    # Data 불러오기
    df_train = train_list[fold_index]
    df_val = val_list[fold_index]

    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)

    train_data = CompDataset(df_train)
    val_data = CompDataset(df_val)

    train_dataloader = torch.utils.data.DataLoader(train_data,
                                                   batch_size=BATCH_SIZE,
                                                   shuffle=True,
                                                   num_workers=4)
    val_dataloader = torch.utils.data.DataLoader(val_data,
                                                 batch_size=BATCH_SIZE,
                                                 shuffle=True,
                                                 num_workers=4)
    # Training
    stacked_val_labels=[]
    targets_list=[]

    print('Training...')
    model.train()
    torch.set_grad_enabled(True)

    total_train_loss=0

    for i, batch in enumerate(train_dataloader):
      train_status = 'Batch' + str(i+1) + 'of' + str(len(train_dataloader))

      print(train_status, end='\r')

      b_input_ids = batch[0].to(device)
      b_attention_mask = batch[1].to(device)
      b_token_type_ids = batch[2].to(device)
      b_labels = batch[3].to(device)

      model.zero_grad()

      outputs = model(input_ids=b_input_ids, attention_mask=b_attention_mask,
                      token_type_ids=b_token_type_ids, labels=b_labels)
      
      loss=outputs[0]
      total_train_loss = total_train_loss + loss.item()
      
      optimizer.zero_grad()
      loss.backward()
      
      # Clip the norm of the gradients to 1.0.
      # This is to help prevent the "exploding gradients" problem.
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      # Optimizer for GPU
      # opimizer.step()

      # Optimizer for TPU
      xm.optimizer_step(optimizer=optimizer, barrier=True)

    print('Train loss:', total_train_loss)

      
    # Validation
    print('\nValidation...')
    model.eval()
      
    torch.set_grad_enabled(False)
    total_val_loss=0

    for j, val_batch in enumerate(val_dataloader):
      val_status = 'Batch' + str(j+1) + 'of' + str(len(val_dataloader))
      print(val_status, end='\r')

      input_ids = val_batch[0].to(device)
      attention_mask = val_batch[1].to(device)
      token_type_ids = val_batch[2].to(device)
      labels = val_batch[3].to(device)

      outputs = model(input_ids=input_ids,
                      attention_mask=attention_mask,
                      token_type_ids=token_type_ids,
                      labels=labels)
        
      loss = outputs[0]
      total_val_loss = total_val_loss + loss.item()
        
      preds = outputs[1]
      val_preds = preds.detach().cpu().numpy()
      target_np = labels.to('cpu').numpy()

      targets_list.extend(target_np)

      if j == 0:  # first batch
        stacked_val_preds = val_preds
      else:
        stacked_val_preds = np.vstack((stacked_val_preds, val_preds))

      
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)

    val_acc = accuracy_score(y_true, y_pred)

    epoch_acc_scores_list.append(val_acc)

    print('Val Loss:', total_val_loss)
    print('Val Acc:', val_acc)

    # save the best model
    if epoch == 0:
      model_name = 'model_' + str(fold_index) + '.bin'
      torch.save(model.state_dict(), model_name)
      print('Saved model as ', model_name)

    if epoch != 0:
      val_acc_list = fold_val_acc_list[fold_index]
      best_val_acc = max(val_acc_list)

      if val_acc > best_val_acc:
        model_name = 'model' + str(fold_index) + '.bin'
        torch.save(model.state_dict(), model_name)
        print('Val acc improved. Saved model as ', model_name)
      
    # save the best fold
    fold_val_acc_list[fold_index].append(val_acc)
    gc.collect()  # save memory

  

  cv_acc = sum(epoch_acc_scores_list) / NUM_FOLDS_TO_TRAIN
  print('\nCV Acc:', cv_acc)

    


Num folds used for training: 3
=========== Epoch 1 / 3 ===========

== Fold Model 0


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training...
Train loss: 311.2814366221428

Validation...
Val Loss: 68.91873496770859
Val Acc: 0.58003300330033
Saved model as  model_0.bin

== Fold Model 1


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training...
Train loss: 315.5517609119415

Validation...
Val Loss: 69.94172239303589
Val Acc: 0.5726072607260726
Saved model as  model_1.bin

== Fold Model 2


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training...
Train loss: 317.5435943007469

Validation...
Val Loss: 73.6936839222908
Val Acc: 0.5346534653465347
Saved model as  model_2.bin

CV Acc: 0.5624312431243125

Num folds used for training: 3
=========== Epoch 2 / 3 ===========

== Fold Model 0
Training...
Train loss: 254.46639397740364

Validation...
Val Loss: 65.25186729431152
Val Acc: 0.610973597359736
Val acc improved. Saved model as  model0.bin

== Fold Model 1
Training...
Train loss: 260.14059668779373

Validation...
Val Loss: 63.2266560792923
Val Acc: 0.6283003300330033
Val acc improved. Saved model as  model1.bin

== Fold Model 2
Training...
Train loss: 271.3419270515442

Validation...
Val Loss: 63.8719579577446
Val Acc: 0.6303630363036303
Val acc improved. Saved model as  model2.bin

CV Acc: 0.6232123212321232

Num folds used for training: 3
=========== Epoch 3 / 3 ===========

== Fold Model 0
Training...
Train loss: 255.4277435541153

Validation...
Val Loss: 64.41935187578201
Val Acc: 0.6179867986798679
Val acc improv

# Test set prediction까지 받아내기(필사)
# https://www.kaggle.com/vbookshelf/basics-of-bert-and-xlm-roberta-pytorch#Section-3

In [50]:
seed_val = 1024

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)



# Store the accuracy scores for each fold model in this list.
# [[model_0 scores], [model_1 scores], [model_2 scores], [model_3 scores], [model_4 scores]]
# [[ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...]]

# Create a list of lists to store the val acc results.
# The number of items in this list will correspond to
# the number of folds that the model is being trained on.
fold_val_acc_list = []
for i in range(0, NUM_FOLDS):
    
    # append an empty list
    fold_val_acc_list.append([])
    
    
    
    

# For each epoch...
for epoch in range(0, NUM_EPOCHS):
    
    print("\nNum folds used for training:", NUM_FOLDS_TO_TRAIN)
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    
    # Get the number of folds
    num_folds = len(train_list)

    # For this epoch, store the val acc scores for each fold in this list.
    # We will use this list to calculate the cv at the end of the epoch.
    epoch_acc_scores_list = []
    
    # For each fold...
    for fold_index in range(0, NUM_FOLDS_TO_TRAIN):
        
        print('\n== Fold Model', fold_index)
        
        
        # .........................
        # Load the fold model
        # .........................
        
        if epoch == 0:
            
            # define the model
            model = BertForSequenceClassification.from_pretrained(
            MODEL_TYPE, 
            num_labels = 3,       
            output_attentions = False, 
            output_hidden_states = False,
            )
            
            # Send the model to the device.
            model.to(device)
            
            optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8
            )
            
        else:
        
            # Get the fold model
            path_model = 'model_' + str(fold_index) + '.bin'
            model.load_state_dict(torch.load(path_model))

            # Send the model to the device.
            model.to(device)
        
        
        
        # .....................................
        # Set up the train and val dataloaders
        # .....................................
        
        
        # Intialize the fold dataframes
        df_train = train_list[fold_index]
        df_val = val_list[fold_index]
        
        # Reset the indices or the dataloader won't work.
        df_train = df_train.reset_index(drop=True)
        df_val = df_val.reset_index(drop=True)
    
        # Create the dataloaders
        train_data = CompDataset(df_train)
        val_data = CompDataset(df_val)

        train_dataloader = torch.utils.data.DataLoader(train_data,
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                               num_workers=4)

        val_dataloader = torch.utils.data.DataLoader(val_data,
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                               num_workers=4)
    
    
    

       

        # ========================================
        #               Training
        # ========================================
        
        stacked_val_labels = []
        targets_list = []

        print('Training...')

        # put the model into train mode
        model.train()

        # This turns gradient calculations on and off.
        torch.set_grad_enabled(True)


        # Reset the total loss for this epoch.
        total_train_loss = 0

        for i, batch in enumerate(train_dataloader):

            train_status = 'Batch ' + str(i+1) + ' of ' + str(len(train_dataloader))

            print(train_status, end='\r')


            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_token_type_ids = batch[2].to(device)
            b_labels = batch[3].to(device)

            model.zero_grad()        


            outputs = model(b_input_ids, 
                        token_type_ids=b_token_type_ids, 
                        attention_mask=b_input_mask,
                        labels=b_labels)

            # Get the loss from the outputs tuple: (loss, logits)
            loss = outputs[0]

            # Convert the loss from a torch tensor to a number.
            # Calculate the total loss.
            total_train_loss = total_train_loss + loss.item()

            # Zero the gradients
            optimizer.zero_grad()

            # Perform a backward pass to calculate the gradients.
            loss.backward()
            
            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Use the optimizer to update Weights
            
            # Optimizer for GPU
            # optimizer.step() 
            
            # Optimizer for TPU
            # https://pytorch.org/xla/
            xm.optimizer_step(optimizer, barrier=True)
            
           


        print('Train loss:' ,total_train_loss)


        # ========================================
        #               Validation
        # ========================================

        print('\nValidation...')

        # Put the model in evaluation mode.
        model.eval()

        # Turn off the gradient calculations.
        # This tells the model not to compute or store gradients.
        # This step saves memory and speeds up validation.
        torch.set_grad_enabled(False)


        # Reset the total loss for this epoch.
        total_val_loss = 0


        for j, val_batch in enumerate(val_dataloader):

            val_status = 'Batch ' + str(j+1) + ' of ' + str(len(val_dataloader))

            print(val_status, end='\r')

            b_input_ids = val_batch[0].to(device)
            b_input_mask = val_batch[1].to(device)
            b_token_type_ids = val_batch[2].to(device)
            b_labels = val_batch[3].to(device)      


            outputs = model(b_input_ids, 
                    token_type_ids=b_token_type_ids, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)

            # Get the loss from the outputs tuple: (loss, logits)
            loss = outputs[0]

            # Convert the loss from a torch tensor to a number.
            # Calculate the total loss.
            total_val_loss = total_val_loss + loss.item()

            # Get the preds
            preds = outputs[1]


            # Move preds to the CPU
            val_preds = preds.detach().cpu().numpy()

            # Move the labels to the cpu
            targets_np = b_labels.to('cpu').numpy()

            # Append the labels to a numpy list
            targets_list.extend(targets_np)

            if j == 0:  # first batch
                stacked_val_preds = val_preds

            else:
                stacked_val_preds = np.vstack((stacked_val_preds, val_preds))
                
                
                
        # .........................................
        # Calculate the val accuracy for this fold
        # .........................................      


        # Calculate the validation accuracy
        y_true = targets_list
        y_pred = np.argmax(stacked_val_preds, axis=1)

        val_acc = accuracy_score(y_true, y_pred)
        
        
        epoch_acc_scores_list.append(val_acc)


        print('Val loss:' ,total_val_loss)
        print('Val acc: ', val_acc)
        
        
        # .........................
        # Save the best model
        # .........................
        
        if epoch == 0:
            
            # Save the Model
            model_name = 'model_' + str(fold_index) + '.bin'
            torch.save(model.state_dict(), model_name)
            print('Saved model as ', model_name)
            
        if epoch != 0:
        
            val_acc_list = fold_val_acc_list[fold_index]
            best_val_acc = max(val_acc_list)
            
            if val_acc > best_val_acc:
                # save the model
                model_name = 'model_' + str(fold_index) + '.bin'
                torch.save(model.state_dict(), model_name)
                print('Val acc improved. Saved model as ', model_name)

            
                
                
                
        # .....................................
        # Save the val_acc for this fold model
        # .....................................
        
        # Note: Don't do this before the above 'Save Model' code or 
        # the save model code won't work. This is because the best_val_acc will
        # become current val accuracy.
                
        # fold_val_acc_list is a list of lists.
        # Each fold model has it's own list corresponding to the fold index.
        # Here we choose a list corresponding to the fold number and append the acc score to that list.
        fold_val_acc_list[fold_index].append(val_acc)
        
            

        # Use the garbage collector to save memory.
        gc.collect()


    
        
        
    # .............................................................
    # Calculate the CV accuracy score over all folds in this epoch
    # .............................................................   
        
        
    # Print the average val accuracy for all 5 folds
    cv_acc = sum(epoch_acc_scores_list)/NUM_FOLDS_TO_TRAIN
    print("\nCV Acc:", cv_acc)
    


Num folds used for training: 3
======== Epoch 1 / 3 ========

== Fold Model 0


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training...


Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, b

KeyboardInterrupt: ignored